In [1]:
cd ../../src/

/Users/in-divye.singh/Documents/Projects/MIC_predictor/src


In [2]:
import biovec
import numpy as np
import pandas as pd
from itertools import chain, combinations
from collections import Counter

from utils import *

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [3]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [4]:
def pearson_score(y_true, y_pred):
    pcc = pearsonr(y_true, y_pred)
    return pcc[0]

pcc_scorer = make_scorer(pearson_score)

In [5]:
avp_ic50 = pd.read_csv("../data/raw/AVP-IC50Pred_train.csv")
ha_avp = pd.read_csv("../data/raw/HA_AVP.csv")

In [6]:
df = pd.concat([avp_ic50[['Sequence','MIC']], ha_avp], axis=0).drop_duplicates(['Sequence']).reset_index(drop=True)
df = sequence_filtering(df)

In [7]:
df['pIC50'] = df['MIC'].apply(lambda x: -np.log(x*1e-6))

In [8]:
family = pd.read_csv("../data/raw/712pep_family.csv")

In [9]:
family['Family'].unique()

array(['Arenaviridae', 'Arteriviridae', 'Asfarviridae', 'Bunyaviridae',
       'Coronaviridae', 'Family', 'Filoviridae', 'Flaviviridae',
       'Hepadnaviridae', 'Herpesviridae', 'Orthomyxoviridae',
       'Papillomaviridae', 'Paramyxoviridae', 'Polyomaviridae',
       'Poxviridae', 'Retroviridae'], dtype=object)

In [10]:
flaviviridae_seq = family[(family['Family'] == "Flaviviridae")].reset_index(drop=True)

In [11]:
flaviviridae_seq

,Sequence,Abbreviation,Family
0,AAQRRGRVGRNPNQVGD,HCV,Flaviviridae
1,RNPSQVGD,WNV,Flaviviridae
2,RVGRNPNQVGD,HCV,Flaviviridae
3,AAQRRGRIGRNPSQVGD,HCV,Flaviviridae
4,RGRRGIYR,HCV,Flaviviridae
...,...,...,...
125,TWLRAIWDWVCTALTDFK,HCV,Flaviviridae
126,SWLRDVWDWVCTVLSDFK,HCV,Flaviviridae
127,GAIVSTALPQWRIYSYAG,HCV,Flaviviridae
128,SWLRDIWDWLCELLSDFK,HCV,Flaviviridae


In [12]:
df_flaviviridae = df.merge(flaviviridae_seq,how='right',on='Sequence').reset_index(drop=True)

In [13]:
df_flaviviridae#.to_csv("../data/raw/flaviviridae_data.csv", index=False)

,Sequence,MIC,pIC50,Abbreviation,Family
0,AAQRRGRVGRNPNQVGD,442.00,7.724201,HCV,Flaviviridae
1,RNPSQVGD,383.00,7.867476,WNV,Flaviviridae
2,RVGRNPNQVGD,374.00,7.891255,HCV,Flaviviridae
3,AAQRRGRIGRNPSQVGD,358.00,7.934978,HCV,Flaviviridae
4,RGRRGIYR,313.00,8.069307,HCV,Flaviviridae
...,...,...,...,...,...
125,TWLRAIWDWVCTALTDFK,7.10,11.855416,HCV,Flaviviridae
126,SWLRDVWDWVCTVLSDFK,3.50,12.562748,HCV,Flaviviridae
127,GAIVSTALPQWRIYSYAG,23.80,10.645825,HCV,Flaviviridae
128,SWLRDIWDWLCELLSDFK,0.82,14.013961,HCV,Flaviviridae


In [14]:
def get_physicochemical_properties(df):
    params = ['aromaticity', 'helix', 'turn', 'sheet', 'gravy', 'net_charge_at_pH7point4']

    prop = []
    for seq in df.Sequence:
        X = ProteinAnalysis(seq)
        aromaticity = X.aromaticity()
        sec_struc = X.secondary_structure_fraction()
        helix = sec_struc[0]
        turn = sec_struc[1]
        sheet = sec_struc[2]
        gravy = X.gravy() # hydrophobicity related
        net_charge_at_pH7point4 = X.charge_at_pH(7.4)

        prop.append([aromaticity, helix, turn, sheet, gravy, net_charge_at_pH7point4])
    return pd.DataFrame(prop, columns=params)

In [15]:
aa_freq = reduce_by_kmer_frequency(df_flaviviridae)

In [16]:
uniprot_embedding = biovec.models.load_protvec("../data/embeddings/uniprot__kmer_3_contextWindow_10_vector_100_reduction_None")

avg_protvec = convert_sequences_to_avg_vectors(df_flaviviridae['Sequence'], uniprot_embedding, kmer=3)
avg_protvec = avg_protvec.reset_index(drop=True)

/Users/in-divye.singh/anaconda3/envs/life-science/lib/python3.6/site-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Creating vectors: 100%|██████████| 130/130 [00:00<00:00, 918.75sequence/s]


In [17]:
physicochemical_prop = get_physicochemical_properties(df_flaviviridae)

In [18]:
X = pd.concat([aa_freq, avg_protvec, physicochemical_prop[['helix','turn','sheet']]], axis=1)

In [19]:
y = df_flaviviridae[['pIC50', 'MIC']]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']

In [22]:
from sklearn.model_selection import cross_val_score, GridSearchCV, LeaveOneOut

In [23]:
from scipy.stats import pearsonr

In [67]:
def multi_objective_score(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    std_diff = abs(np.std(y_pred) - np.std(y_true))
    pcc = pearson_score(y_true, y_pred)
    return mape - 10*pcc + 20*std_diff
multi_objective_scorer = make_scorer(multi_objective_score, greater_is_better=False)

In [68]:
param_grid = {
    'C':[0.001,0.01,0.1,1,10,100,1000],
    'kernel':['rbf','poly','sigmoid','linear'],
    'degree':[1,2,3,4,5,6],
    'gamma': np.arange(0.1,1.0,0.1).round(1).tolist() + np.arange(1,11,1).round().tolist()
}
svr = SVR()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = svr, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, scoring=multi_objective_scorer)

In [69]:
grid_search.fit(X_train, y_train_pmic)

Fitting 5 folds for each of 3192 candidates, totalling 15960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 3560 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 8088 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 13928 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 15960 out of 15960 | elapsed:   31.2s finished


GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6],
                         'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                                   1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             scoring=make_scorer(multi_objective_score, greater_is_better=False),
             verbose=2)

In [70]:
grid_search.best_params_

{'C': 100, 'degree': 2, 'gamma': 5, 'kernel': 'poly'}

In [71]:
best_grid = grid_search.best_estimator_

In [72]:
y_pred_pmic = best_grid.predict(X_test)

In [73]:
np.std(y_test_pmic), np.std(y_pred_pmic)

(2.4649130950912346, 2.3991496738662024)

In [74]:
mean_absolute_percentage_error(y_test_pmic, y_pred_pmic)

21.85028579451872

In [54]:
y_pred_mic = np.exp(-y_pred_pmic)/1e-6

In [55]:
ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic

In [56]:
mean_absolute_percentage_error(y_test_mic, y_pred_mic)

662.8941457346394

In [57]:
pearson_score(y_test_mic, y_pred_mic)

0.03010366865023491

In [58]:
list(zip(y_test_mic.round(4), y_pred_mic.round(4), ape_mic))

[(8.9, 0.4834, 94.56862307274307),
 (12.5, 4.7519, 61.984938303068205),
 (27.0, 3.0777, 88.60126965919842),
 (21.4, 454.4669, 2023.677307915668),
 (0.024, 1.1322, 4617.562065526883),
 (8.0, 3.7138, 53.577385905179376),
 (5.0, 2.3292, 53.41528544372768),
 (0.51, 8.9141, 1647.8675027325946),
 (3.0, 5.3323, 77.74319588966948),
 (25.0, 0.1883, 99.24661132158954),
 (1.115, 0.4823, 56.742702747316166),
 (25.0, 0.754, 96.98408578036188),
 (5.1, 2.6347, 48.34011089144721),
 (313.0, 0.0291, 99.99071694992341),
 (0.89, 0.3193, 64.12485560795342),
 (0.8, 4.023, 402.8694340152281),
 (13.5, 1.4244, 89.44921016337194),
 (3.0, 4.0449, 34.83147282932449),
 (1.2, 0.4899, 59.17613771203687),
 (1.965, 0.005, 99.74556608491423),
 (27.0, 2.4627, 90.8790483830435),
 (36.0, 571.3364, 1487.0455839868227),
 (0.001, 0.0565, 5546.032975356189),
 (34.6, 0.7854, 97.7301989446528),
 (3.5, 0.7584, 78.33107441682088),
 (10.0, 3.527, 64.73042946090007)]

In [59]:
loo = LeaveOneOut()

from tqdm import tqdm

result_df = pd.DataFrame(columns = list(df_flaviviridae.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
for train_index, test_index in tqdm(loo.split(X)):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
    y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']
    svr = SVR(**grid_search.best_params_)
    _ = svr.fit(X_train, y_train_pmic)
    y_pred_pmic = svr.predict(X_test)
    y_pred_mic = np.exp(-y_pred_pmic)/1e-6
    ape = 100*np.abs(y_test_pmic-y_pred_pmic)/y_test_pmic
    ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic
    df_val = df_flaviviridae.iloc[test_index,:].values[0].tolist()
    res = np.append(df_val, [y_pred_pmic[0], y_pred_mic[0], ape.values[0], ape_mic.values[0]])
    res = pd.DataFrame([res], columns = list(df_flaviviridae.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
    result_df = result_df.append(res)
result_df = result_df[["Sequence", "pIC50", "y_pred_pmic", "ape_pmic", "MIC", "y_pred_mic", "ape_mic"]]

130it [00:02, 55.82it/s]


In [60]:
result_df#.to_csv("../results/SVM_HIV_CoV_pMIC_to_MIC_rbf_c_100_gamma_2.csv", index=False)

,Sequence,pIC50,y_pred_pmic,ape_pmic,MIC,y_pred_mic,ape_mic
0,AAQRRGRVGRNPNQVGD,7.724200675886576,8.28316039447111,7.2364732875143485,442.0,252.73718731329384,42.819640879345286
0,RNPSQVGD,7.867475568783628,7.778131169309699,1.1356171200381937,383.0,418.79409900642503,9.345717756246744
0,RVGRNPNQVGD,7.891254760549742,6.957773366905505,11.829315133899021,374.0,951.2122188913861,154.33481788539734
0,AAQRRGRIGRNPSQVGD,7.934977571563574,8.18344724890725,3.131321734721865,358.0,279.2376772787253,22.000648804825335
0,RGRRGIYR,8.069307367424122,15.446827452628037,91.4269310769725,313.0,0.1956717253333821,99.93748507177847
...,...,...,...,...,...,...,...
0,TWLRAIWDWVCTALTDFK,11.855415773917004,10.063638368474797,15.113577116243203,7.1,42.60076408957533,500.01076182500464
0,SWLRDVWDWVCTVLSDFK,12.562747589468906,12.86820716661365,2.431471101121269,3.5,2.5787464050891025,26.3215312831685
0,GAIVSTALPQWRIYSYAG,10.645824977286845,9.352683715528082,12.146933323793265,23.8,86.73234154558335,264.4216031327032
0,SWLRDIWDWLCELLSDFK,14.013961496688113,13.303024514001473,5.073062194830876,0.82,1.6694363324295065,103.58979663774471


In [61]:
result_df['ape_pmic'].astype('float').mean()

15.031607822815511

In [62]:
result_df['ape_mic'].astype('float').mean()

975.364340874116

In [63]:
pearsonr(result_df['MIC'].astype('float'), result_df['y_pred_mic'].astype('float'))

(0.6008593235527914, 4.10821793382307e-14)

### Train with MIC

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [65]:
y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']

In [66]:
max_n_neighbors = int(np.sqrt(X_train.shape[0]))
param_grid = {
    'n_neighbors': range(1, max_n_neighbors),
    'weights': ['uniform', 'distance'],
    'metric': ["euclidean", "manhattan", "chebyshev"]
}
knn = KNeighborsRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = knn, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2, scoring=mape_scorer)

In [ ]:
grid_search.fit(X_train, y_train_mic)

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

In [ ]:
y_pred_mic = best_grid.predict(X_test)

In [ ]:
mean_absolute_percentage_error(y_test_mic, y_pred_mic)

In [ ]:
len(y_test_mic), len(y_pred_mic)

In [ ]:
pearson_score(y_test_mic, y_pred_mic)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test_mic, y_pred_mic)

In [ ]:
ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic

In [ ]:
list(zip(y_test_mic.round(4), y_pred_mic.round(4), ape_mic))

In [ ]:
loo = LeaveOneOut()

from tqdm import tqdm

result_df = pd.DataFrame(columns = list(df_flaviviridae.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
for train_index, test_index in tqdm(loo.split(X)):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    y_train_pmic, y_train_mic = y_train['pIC50'], y_train['MIC']
    y_test_pmic, y_test_mic = y_test['pIC50'], y_test['MIC']
    knn = KNeighborsRegressor(**grid_search.best_params_)
    _ = knn.fit(X_train, y_train_pmic)
    y_pred_pmic = knn.predict(X_test)
    y_pred_mic = np.exp(-y_pred_pmic)/1e-6
    ape = 100*np.abs(y_test_pmic-y_pred_pmic)/y_test_pmic
    ape_mic = 100*np.abs(y_test_mic-y_pred_mic)/y_test_mic
    df_val = df_flaviviridae.iloc[test_index,:].values[0].tolist()
    res = np.append(df_val, [y_pred_pmic[0], y_pred_mic[0], ape.values[0], ape_mic.values[0]])
    res = pd.DataFrame([res], columns = list(df_flaviviridae.columns)+["y_pred_pmic", "y_pred_mic", "ape_pmic", "ape_mic"])
    result_df = result_df.append(res)
result_df = result_df[["Sequence", "pIC50", "y_pred_pmic", "ape_pmic", "MIC", "y_pred_mic", "ape_mic"]]

In [ ]:
result_df#.to_csv("../results/kNN_flaviviridae_dist_manhattan_k_2_weight_uniform.csv", index=False)

In [ ]:
result_df['ape_pmic'].astype('float').mean()

In [ ]:
result_df['ape_mic'].astype('float').mean()

In [ ]:
pearsonr(result_df['MIC'].astype('float'), result_df['y_pred_mic'].astype('float'))

In [ ]:
r2_score(result_df['MIC'].astype('float'), result_df['y_pred_mic'].astype('float'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.relplot(x=result_df['pIC50'].astype('float'), y=result_df['y_pred_pmic'].astype('float'))

In [ ]:
sns.relplot(x=result_df['y_pred_pmic'].astype('float'), y=result_df['pIC50'].astype('float')-result_df['y_pred_pmic'].astype('float'))